In [5]:
import sys
sys.path.append('../')

from lxml import etree
import abc
import re
import requests

from gensim.models.phrases import *

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import searchbetter.search as search
reload(search)
import searchbetter.rewriter as rewriter
reload(rewriter)


<module 'searchbetter.rewriter' from '../searchbetter/rewriter.pyc'>

In [8]:
corpus = word2vec.Text8Corpus(secure.DATASET_PATH_BASE + 'enwik8')
# self.model = word2vec.Word2Vec(corpus, workers=8)
# self.model.save(self.MODEL_PATH)

In [ ]:


ph = Phrases(corpus)

In [ ]:
phr = Phraser(ph)

In [ ]:
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']

In [ ]:
print phr[sent]

In [ ]:
phr[[u'chemical',u'reactions',u'are',u'cool']]

In [ ]:
import warnings

In [ ]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [ ]:
mod = word2vec.Word2Vec(ph[corpus],size=10)

In [ ]:
mod.most_similar(positive=['person'], topn=10)

In [ ]:
from gensim.models.wrappers import FastText

In [ ]:
# WARNING: takes like 10 min to train
# made from https://github.com/facebookresearch/fastText
path = '/Users/neel/Dropbox/Harvard/Classes/research/query-rewriter/fastText/fasttext'
fm = FastText.train(path, corpus_file=secure.DATASET_PATH_BASE + 'wikiclean8')

In [ ]:
fm.similar_by_word('new_york', topn=20)

In [ ]:
from gensim.models.wrappers import FastText
import secure

In [ ]:
fm.most_similar(positive=['new_york'], topn=20)

In [ ]:
fm.save(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [ ]:
fm2 = FastText.load(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [ ]:
fm2.similar_by_word('chemistry', topn=25)

In [ ]:
fm2.init_ngrams()

In [ ]:
corpus = word2vec.LineSentence(secure.DATASET_PATH_BASE + 'wikiclean8')

w2v = rewriter.Word2VecRewriter(corpus=None, create=False)

In [ ]:
w2v.rewrite("ohio")

In [ ]:
from gensim.models.phrases import *

phrases = Phrases(corpus)

In [ ]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [ ]:
model_bigram = word2vec.Word2Vec(phrases[corpus])

In [ ]:
# warning: you get a lot of gibberish! :(
model_bigram.similar_by_word("new_york")

In [ ]:
model = word2vec.Word2Vec.load(MODEL_PATH)

In [ ]:
model.similar_by_word("learning")

In [ ]:
phraser = Phraser(phrases)

In [ ]:
phraser["this is a chemical reaction".split(" ")]

In [ ]:
import collections

z = 0

class PhraserCorpus(collections.Iterator):
    """
    An iterable that wraps over a Corpus and the Phraser. When word2vec
    requests a sentence from the corpus, this passes the sentence through the
    Phraser first.
    
    This makes it very easy to implement bigrams with just a vanilla Corpus
    and a simple Phraser.
    """
    def __init__(self, phraser, corpus_iter):
        self.corpus_iter = corpus_iter
        self.phraser = phraser
        
    def __iter__(self): return self
    
    def next(self):
        # wrap the corpus_iter.next() call with a call to the phraser 
        # if it throws a stop iteration
        try:
            sentence = self.corpus_iter.next()
            global z
            z += 1
            # apply the phraser here 
            return phraser[sentence]
        except StopIteration:
            # we should stop when it stops
            raise StopIteration

In [ ]:
ci = iter(corpus)

In [ ]:
pc = PhraserCorpus(phraser, ci)

In [ ]:
mod3 = word2vec.Word2Vec(pc)

In [ ]:
mode3

In [ ]:
mod3.similar_by_word("french_revolution")

In [ ]:
import collections

class ControlCorpus(collections.Iterator):
    """
    An iterable that wraps over a Corpus and the Phraser. When word2vec
    requests a sentence from the corpus, this passes the sentence through the
    Phraser first.
    
    This makes it very easy to implement bigrams with just a vanilla Corpus
    and a simple Phraser.
    """
    def __init__(self, phraser, corpus_iter):
        self.corpus_iter = corpus_iter
        self.phraser = phraser
        
    def __iter__(self):
        try:
            while True:
                yield self.corpus_iter.next()
    
#     def next(self):
#         # wrap the corpus_iter.next() call with a call to the phraser 
#         # if it throws a stop iteration
#         try:
#             sentence = self.corpus_iter.next()
#             global z
#             z += 1
#             # apply the phraser here 
#             return sentence
#         except StopIteration:
#             # we should stop when it stops
#             raise StopIteration

In [ ]:
ci2 = iter(corpus)

In [ ]:
pd = ControlCorpus(phraser, ci2)

In [ ]:
mod4 = word2vec.Word2Vec(pd)

In [ ]:
mod4.similar_by_word("chemistry")

In [ ]:
w2v.model.similar_by_word("chemistry")

In [189]:
phraser

In [190]:
corpus

In [191]:
ii = phraser[corpus]

In [192]:
ii

In [193]:
mod5 = word2vec.Word2Vec(ii)

In [194]:
mod5.similar_by_word("chemistry")

[(u'bipartisan', 0.40629392862319946),
 (u'strict_observance', 0.3852868974208832),
 (u'became_estranged', 0.37987571954727173),
 (u'juliet_barker', 0.3734053373336792),
 (u'team_finished', 0.37240421772003174),
 (u'motor_neuron', 0.3719816207885742),
 (u'thunder_bay', 0.3712693750858307),
 (u'mes', 0.3698464632034302),
 (u'contentious_issue', 0.3677879571914673),
 (u'uprisings', 0.3670933246612549)]

## Using a phraser doesn't work... sometimes

Word2Vec has a feature called a Phraser, which generates a corpus of bigrams out of text. But oddly, sometimes it just doesn't work. For instance, here's a toy example. First, we use the raw corpus and a normal word2vec model, and everything works fine...

In [7]:
corpus = word2vec.LineSentence(secure.DATASET_PATH_BASE + 'wikiclean8')

In [30]:
raw_model = word2vec.Word2Vec(corpus)

In [33]:
raw_model.save(secure.MODEL_PATH_BASE + 'testw2v/testw2v')

In [2]:
raw_model = word2vec.Word2Vec.load(secure.MODEL_PATH_BASE + 'testw2v/testw2v')

In [48]:
raw_model.similar_by_word("chemistry", topn=25)

[(u'biochemistry', 0.8630732297897339),
 (u'biology', 0.8516983985900879),
 (u'physics', 0.8163505792617798),
 (u'molecular', 0.7827188968658447),
 (u'synthesis', 0.7580161690711975),
 (u'mathematics', 0.7579576969146729),
 (u'computational', 0.7358349561691284),
 (u'optics', 0.7268904447555542),
 (u'botany', 0.726080596446991),
 (u'zoology', 0.7202526330947876),
 (u'theoretical', 0.716191291809082),
 (u'magnetism', 0.7125887870788574),
 (u'geometry', 0.7071623802185059),
 (u'mechanics', 0.705739438533783),
 (u'nanotechnology', 0.703361988067627),
 (u'genetics', 0.7015490531921387),
 (u'anatomy', 0.6992816925048828),
 (u'astronomy', 0.6973481178283691),
 (u'inorganic', 0.6922664642333984),
 (u'physiology', 0.6913220882415771),
 (u'psychology', 0.6904988288879395),
 (u'pathology', 0.6902084350585938),
 (u'embryology', 0.6896854639053345),
 (u'medicine', 0.688435971736908),
 (u'linguistics', 0.6849200129508972)]

In [14]:
lc = list(corpus)

Now, if we try using the Phraser class, we get total garbage (observe the drop from 99.97% relevance to 32%):

In [8]:
phrases = Phrases(corpus)

In [12]:
list(phrases.export_phrases([u"chemical",u"reactions",u"are",u"fun"], as_tuples=True))

[(('i', 'o'), 10.3886122692534)]

In [17]:
phrases[u"chemical",u"reactions",u"are",u"fun"]

/Users/neel/usr/anaconda2/lib/python2.7/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[u'chemical_reactions', u'are', u'fun']

In [18]:
phraser = Phraser(phrases)

In [19]:
phraser.save(secure.MODEL_PATH_BASE + 'phr2/phr2')

In [23]:
list(phraser[[u"chemical",u"reactions",u"are",u"fun"],[u"i",u"have",u"a",u"cognitive",u"bias"]])

[[u'chemical_reactions', u'are', u'fun'],
 [u'i', u'have', u'a', u'cognitive_bias']]

In [24]:
bigrammed_model = word2vec.Word2Vec(phraser[corpus])

In [27]:
bigrammed_model.similar_by_vector("chemical_reaction")

[(u'unocal', 0.4329361021518707),
 (u'midwife', 0.40027716755867004),
 (u'hemiparesis', 0.39055171608924866),
 (u'lili', 0.38215988874435425),
 (u'mu_bai', 0.3786051273345947),
 (u'angioplasty', 0.3718985915184021),
 (u'fujiko', 0.36773407459259033),
 (u'seventy', 0.36423635482788086),
 (u'aesthetically', 0.35747605562210083),
 (u'probability_distribution', 0.355782151222229)]

Even stranger still - if we convert the  phraser into a list and then feed it into the model, the model works OK and almost the same as before. But why???

In [28]:
phrase_list = list(phraser[corpus])

In [29]:
listed_model = word2vec.Word2Vec(phrase_list)

In [ ]:
raw_model.save(secure.MODEL_PATH_BASE + 'testw2v/testw2v')

In [47]:
listed_model.similar_by_word("chemical_reaction", topn=25)

[(u'condensation', 0.8774197101593018),
 (u'enzyme', 0.8755686283111572),
 (u'plasma', 0.8648638725280762),
 (u'glycolysis', 0.8633915781974792),
 (u'photon', 0.8597602844238281),
 (u'molecule', 0.8568265438079834),
 (u'fission', 0.8560503721237183),
 (u'emission', 0.8546547889709473),
 (u'reactant', 0.8438876271247864),
 (u'mutation', 0.8415351510047913),
 (u'pathway', 0.8414035439491272),
 (u'proton', 0.8413915038108826),
 (u'decomposition', 0.8393369913101196),
 (u'substrate', 0.8379263877868652),
 (u'hydrolysis', 0.8374930620193481),
 (u'thermodynamic', 0.8366049528121948),
 (u'atp', 0.8346669673919678),
 (u'fluid', 0.8336173295974731),
 (u'bonding', 0.8325698375701904),
 (u'lattice', 0.8315175771713257),
 (u'cyclic', 0.830265998840332),
 (u'catalytic', 0.8297764658927917),
 (u'magnetic_field', 0.8297403454780579),
 (u'electrolysis', 0.8287901282310486),
 (u'high_energy', 0.8285620808601379)]

In [34]:
import cPickle as pickle

In [35]:
with open(secure.MODEL_PATH_BASE + 'pickle/wikpickle.p', 'wb') as file:
    pickle.dump(phrase_list, file)

In [ ]:
modifi